# Merged T cell from BRCA ICI studies
EGAS00001004809 and GSE169246



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import os as os
import seaborn as sns
import sys as sys
sys.path.append('/home/xinghua/projects/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *


## Read in T cells of EGAS100010040809 


In [ ]:
adata_EGAS100010040809 = sc.read_h5ad('/data/ICI_exprs/ICI_T_cell_collection/EGAS00001004809-1863-counts_cells_cohort1_T_cells.h5ad')
print(adata_EGAS100010040809.shape)
print(adata_EGAS100010040809.obs.columns)

In [ ]:
# parse sample id
adata_EGAS100010040809.obs['sample_id'] = ['_'.join(x.split('_')[:3]) for x in adata_EGAS100010040809.obs.index]
adata_EGAS100010040809.obs['sample_id'].value_counts()

In [ ]:
# change the values of timepoint to lower case
adata_EGAS100010040809.obs['timepoint'] = adata_EGAS100010040809.obs['timepoint'].str.lower()

In [ ]:
adata_EGAS100010040809.raw = None

## Read in GSE169246_all.Tcell

In [ ]:
adata_GSE169246 = sc.read('/data/ICI_exprs/ICI_T_cell_collection/GSE169246_T_cells.h5ad')


In [ ]:
print(adata_GSE169246.shape)
print(adata_GSE169246.obs.columns)


### Find tumor-infiltrating and anti-PD-L1 treated patients
GSE169246_all.Tcell samples include both chemo and ICI treated patients
Also monitering time is different for each patient
    Pre: ??
    On  ??
    Progression ??


In [ ]:
# check treatment
print(adata_GSE169246.obs['treatment'].value_counts())
#check sample source
print(adata_GSE169246.obs['sample_source'].value_counts())

### Keep the anti-pd-L1 treated tumor infiltrating cells

In [ ]:
# retain only cells with treatment == 'Anti-PD-L1+ Chemo' and sample_source == 't'
adata_GSE169246_ICI = adata_GSE169246[adata_GSE169246.obs['treatment'] == 'Anti-PD-L1+ Chemo'].copy()
print(adata_GSE169246_ICI.obs['treatment'].value_counts())

adata_GSE169246_ICI = adata_GSE169246_ICI[adata_GSE169246_ICI.obs['sample_source'] == 't'].copy()
print(adata_GSE169246_ICI.obs['sample_source'].value_counts())
print(adata_GSE169246_ICI.shape)

Exclude cells derived from progression because the time of progression is not clear

In [ ]:
print(adata_GSE169246_ICI.obs['timepoint'].value_counts())

# retain only cells with timepoint == 'pre' or 'on'
adata_GSE169246_ICI = adata_GSE169246_ICI[adata_GSE169246_ICI.obs['timepoint'].isin(['pre', 'on'])].copy()
print(adata_GSE169246_ICI.shape)

## Merge two datasets
### Find common genes

In [ ]:
common_genes = np.intersect1d(adata_EGAS100010040809.var_names, adata_GSE169246_ICI.var_names)
print(len(common_genes))

In [ ]:
# extract data for common genes
EGAS100010040809_adata = adata_EGAS100010040809[:, common_genes]
print(EGAS100010040809_adata.shape)
GSE169246_ICI_adata = adata_GSE169246_ICI[:, common_genes]
print(GSE169246_ICI_adata.shape)

In [ ]:
# add a batch column to datasets
GSE169246_ICI_adata.obs['batch'] = ['GSE169246']*len(GSE169246_ICI_adata)
EGAS100010040809_adata.obs['batch'] = ['EGAS100010040809']*len(EGAS100010040809_adata)

In [ ]:
# merge datasets
meta_to_keep = ['patient_id', 'timepoint', 'sample_id', 'batch']
GSE169246_ICI_adata.obs = GSE169246_ICI_adata.obs[meta_to_keep]
GSE169246_ICI_adata.raw = None
EGAS100010040809_adata.obs = EGAS100010040809_adata.obs[meta_to_keep]
EGAS100010040809_adata.raw = None

### Concatenate two datasets into a common 'adata' object

In [ ]:
# concatenate datasets
adata = ad.concat ([EGAS100010040809_adata, GSE169246_ICI_adata])
print(adata.shape)
adata.obs['batch'].value_counts()

## Process combined data


In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

#### Removing cells expressing <500 || >5000 genes:

In [ ]:
# removing cells expressing <500 || >5000 genes
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_cells(adata, max_genes=5000)

In [ ]:
print ('Dimention of adata: ' + str(adata.shape))
print ('columns for adata.obs: ' + str(adata.obs.columns))

#### Removing cells containing <400 || >25000 UMIs:

In [ ]:
# removing cells containing <400 || >25000 UMIs
sc.pp.filter_cells(adata, min_counts = 400)
sc.pp.filter_cells(adata, max_counts = 25000)

In [ ]:
print ('Dimention of adata: ' + str(adata.shape))
print ('columns for adata.obs: ' + str(adata.obs.columns))

### Remove cells with high percentage of mitochondrial genes

In [ ]:
# label genes as mt
adata.var['mt'] = adata.var_names.str.startswith('MT-')  

# annotate cells with the percent of genes assigned as mt
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Here we keep cells with < 20% mito ratio
adata = adata[adata.obs['pct_counts_mt'] < 20, :]
adata.shape

#Plot statistics regarding cells
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.0, multi_panel=True)

## Preprecess with respect to gene (var)

In [ ]:
# removing genes covered by <100 cells
sc.pp.filter_genes(adata, min_cells= 100)
adata.var_names_make_unique()
adata.shape

## Normalizing

#### Normalization & Logarithmization:

In [ ]:
# Log normalization scaled up to 10000
print('Before normalization, the sum of first row of X: ' + str(adata.X[0,:].sum()))
sc.pp.normalize_total(adata, target_sum=1e4)
print('After normalization, the sum of first row of X: ' + str(adata.X[0,:].sum()))


In [ ]:
print('Before log, the sum of first row of X: ' + str(adata.X[0,:].sum()))
# Logarithmize adata
sc.pp.log1p(adata, base=2)
print('After log, the sum of first row of X: ' + str(adata.X[0,:].sum()))

## Keep original data and find high variance genes 

In [ ]:
print ('adata dimensions: ' + str(adata.shape))
adata.raw = adata

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
sc.pl.highly_variable_genes(adata)

### Keep track of original adata and update adata.X to  high variance genes only

In [ ]:
adata = adata[:, adata.var.highly_variable]
print('adata dimensions of high variance genes: ' + str(adata.shape))


In [ ]:
adata.write('/data/ICI_exprs/ICI_NHDP/Merged_GSE169246_EGAS100010040809_T_cell_5K_hvg.h5ad')

In [ ]:
adata = sc.read('/data/ICI_exprs/ICI_NHDP/Merged_GSE169246_EGAS100010040809_T_cell_5K_hvg.h5ad')

## 3. Unsupervised cell clustering & identification of major cell types

In [ ]:
# perform PCA   
sc.tl.pca(adata, svd_solver='arpack', n_comps=50)

In [ ]:
print(adata.obsm['X_pca'].shape)
print(adata.varm['PCs'].shape)
print(adata.uns['pca']['variance_ratio'].shape)
print(adata.obs.columns)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=80, n_pcs=50)

In [ ]:
# Use the Leiden algorithm to find clusters
sc.tl.leiden(adata, resolution=0.5)


In [ ]:
# load/find cell cycle markers: T-test/T-cells
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

## 3. Unsupervised cell clustering & identification of major cell types

In [ ]:
# perform UMAP
sc.tl.umap(adata)
sc.pl.umap(adata, color=['leiden'],legend_loc='on data')

### Check batch effect  

In [ ]:
sc.pl.umap(adata, color=['batch', 'PDCD1', 'CXCL13'])

Cluster #9 appears to be an outlier cluster with cells only from GSE169246_all.Tcell

In [ ]:
# remove cells from cluster #9 from the dataset
# adata = adata[adata.obs['leiden'] != '9', :].copy()

## Perform Harmony analysis
The data above show that there is a significant batch effects. Apply Harmony analysis to re-project cells into new PCA spact and re-perfrom clustering analysis

In [ ]:
# perform batch correction using harmony, which works in the PCA space instead of the original gene space
from scanpy.external.pp import harmony_integrate
sc.external.pp.harmony_integrate(adata, 'batch',  max_iter_harmony=20, random_state=0)


### Clustering again

In [ ]:
# cluster cells again after batch correction
sc.pp.neighbors(adata, n_neighbors=80, n_pcs=50, use_rep='X_pca_harmony')

In [ ]:
sc.tl.leiden(adata, resolution=.5)

In [ ]:
# plot UMAP
sc.tl.umap(adata)
sc.pl.umap(adata, color=['leiden', 'batch', 'timepoint'])

### Save a copy with cluster labels.

In [ ]:
adata.write('/data/ICI_exprs/ICI_NHDP/Merged_GSE169246_EGAS100010040809_T_cell_5K_hvg_with_cls_meta.h5ad')

### Check the distribution of T markers

In [ ]:
# color by gene expression of T cell markers
sc.pl.umap(adata, color= ['CD3D', 'CD3E'])

### Label TNK subtypes

In [ ]:
T_cell_makers = {
    'CD4'	: ['CD4', 'IL7R'],
    'CD8'	: [ 'CD8A', 'CD8B'],
    'Naïve'	: ['TCF7', 'SELL', 'LEF1', 'CCR7'],
    'Exhausted' : ['LAG3', 'TIGIT', 'PDCD1', 'HAVCR2', 'CTLA4'],
    'Cytotoxic' : ['IL2', 'GZMA', 'GNLY', 'PRF1', 'GZMB', 'GZMK', 'IFNG', 'NKG7'],
    'Treg' : ['IL2RA', 'FOXP3', 'IKZF2', 'IKZF4',  'TNFRSF18'],
    'Gamma-delta' : ['TRGC1', 'TRGC2', 'TRDC'],
    'Th17' : [ 'CCR6', 'KLRB1'],  #'IL22',
    'MAIT' : ['SLC4A10', 'KLRB1', 'IL7R', 'DPP4'],  
    'ILC' :	['KIT', 'IL1R1'],
    'Th1' :	['STAT4', 'IL12RB2', 'IFNG'],
    'Th2' :	['GATA3', 'STAT6'],
    'Tfh'	: ['MAF', 'CXCL13', 'CXCR5', 'PDCD1'],
    'NK' :  ['XCL1', 'FCGR3A', 'KLRD1', 'KLRF1', 'NCAM1'],
    'Proliferation' : ['MKI67', 'PCNA', 'STMN1']
}

In [ ]:
# check if the markers are in the var names
for cell_type, markers in T_cell_makers.items():
    print (cell_type, ":", markers)
    print ("number of match in var: ", str(sum(adata.raw.var_names.isin(markers))))

### Plot

In [ ]:
sc.tl.dendrogram(adata, groupby='leiden')
sc.pl.dotplot(adata, T_cell_makers, 'leiden', dendrogram=True)

In [ ]:
for cell_type, markers in T_cell_makers.items():
    print (cell_type, ":", markers)
    sc.pl.umap(adata, color=markers)

In [ ]:
adata.obsm.keys

Plot PD-1 and potential target genes

In [ ]:
sc.pl.umap(adata, color= ['PDCD1', 'CXCL13', 'HAVCR2','CTLA4', "PRDM1"])

The enriched genes for each cluster

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

### Examine the distribution of PDCD1 and potential target genes

In [ ]:
# plot the umap with the timepoint
sc.pl.umap(adata, color=['timepoint', 'PDCD1', 'CXCL13', 'HAVCR2','CTLA4', 'batch'])

### Compare PDCD1 and CXCL13

In [ ]:
# extract cells wtih PDCD1 > 0.5
adata_PDCD1 = adata[adata.raw[:, 'PDCD1'].X > 0.5, :].copy()
# plot violin plot with values > 1
sc.pl.violin(adata_PDCD1, ['PDCD1', 'CXCL13', 'TIGIT', 'HAVCR2'], split=True, groupby='timepoint', jitter=0.0, multi_panel=True)

In [ ]:
sc.pl.violin(adata_PDCD1, ['PDCD1', 'CXCL13', 'TIGIT', 'HAVCR2'], split=True, groupby='batch',  jitter=0.0, multi_panel=True)

It appears that cells expressing these genes exhibit bimodal distribution.  Whether different mode correspond to different cell type is unclear

## Examine the paird samples

In [ ]:
# examine the number of pre and on samples, grouped by patient id, collect number of cells for each patient and timepoint
grouped_by_patient = adata.obs.groupby('patient_id')['timepoint'].value_counts()
# print(grouped_by_patient)
grouped_by_patient


In [ ]:
# iterate through patients and find patients with both pre and on timepoints
patients_with_both_timepoints = []
for patient_id in grouped_by_patient.index.levels[0]:
    #print(patient_id)
    if grouped_by_patient[patient_id]['pre'] > 0 and grouped_by_patient[patient_id]['on'] > 0:
        patients_with_both_timepoints.append(patient_id)
print(patients_with_both_timepoints)
print(len(patients_with_both_timepoints))


### Collect cells from patients with both 'pre' and 'on' timepoints and perform paired t-test
Further extract only cells expressing PDCD1 

In [ ]:
# Collect cells from patients with both timepoints
adata_both_timepoints = adata[adata.obs['patient_id'].isin(patients_with_both_timepoints), :].copy()
adata_both_timepoints_PDCD1 = adata_both_timepoints[adata_both_timepoints.raw[:, 'PDCD1'].X > 0.5, :].copy()

In [ ]:
deg_df = paird_ttest(adata_both_timepoints, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id', pval_cutoff = 0.05, log2fc_cutoff = 1)

In [ ]:
deg_df.head()

In [ ]:
# retein genes with qval < 0. and sort by qval
deg_df = deg_df[deg_df['qval'] < 0.1 ].copy()
deg_df = deg_df.sort_values(by=['qval'])
print(deg_df.shape)
deg_df.head()


### Check if both datasets return samilar deg results

In [ ]:
# Separate EGAS100010040809 and GSE169246
adata_EGAS100010040809_PDCD1 = adata_both_timepoints_PDCD1[adata_both_timepoints_PDCD1.obs['batch'] == 'EGAS100010040809', :].copy()
adata_GSE169246_PDCD1 = adata_both_timepoints_PDCD1[adata_both_timepoints_PDCD1.obs['batch'] == 'GSE169246', :].copy()

deg_df_EGAS100010040809 = paird_ttest(adata_EGAS100010040809_PDCD1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id', pval_cutoff = 0.05, log2fc_cutoff =1)
deg_df_GSE169246 = paird_ttest(adata_GSE169246_PDCD1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id', pval_cutoff = 0.05, log2fc_cutoff = 1)

In [ ]:
# sort through EGA genes
# deg_df_EGAS100010040809 = deg_df_EGAS100010040809[deg_df_EGAS100010040809['qval'] < 0.1 ].copy()
deg_df_EGAS100010040809 = deg_df_EGAS100010040809.sort_values(by=['pval'])
print(deg_df_EGAS100010040809.shape)
deg_df_EGAS100010040809.head()

In [ ]:
# sort through GSE genes
# deg_df_GSE169246 = deg_df_GSE169246[deg_df_GSE169246['qval'] < 0.1 ].copy()
deg_df_GSE169246 = deg_df_GSE169246.sort_values(by=['pval'])
print(deg_df_GSE169246.shape)
deg_df_GSE169246.head()

In [ ]:
# find the location of FBXO34 in EGAS100010040809 index
deg_df_EGAS100010040809.index.get_loc('FBXO34')

# find the location of PRDM1 in GSE169246 index
deg_df_GSE169246.index.get_loc( 'TSC22D3')


### Examine the relationship between PD-1 and potential target genes pre-treatment



In [ ]:
# plot scatter plot for pre 
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(CD274_pre, cxcl13_pre, s=2, color='black')
axes[0].set_xlabel('CD274')
axes[0].set_ylabel('CXCL13')
axes[1].scatter(pdcd1_pre, cxcl13_pre, s=2, color='black')
axes[1].set_xlabel('PDCD1')
# plt.title('Pre')
axes[2].scatter(pdcd1_pre * CD274_pre, cxcl13_pre, s=2, color='black' )
axes[2].set_xlabel('PDCD1 * CD274')
fig.tight_layout()


plt.show()

### Plot relationship on treatment


In [ ]:
# Extract from adata_sample_tpm 
cxcl13_on = adata_sample_tpm.X[adata_sample_tpm.obs['timepoint'] == 'On', adata_sample_tpm.var_names == 'CXCL13'] 
pdcd1_on = adata_sample_tpm.X[adata_sample_tpm.obs['timepoint'] == 'On', adata_sample_tpm.var_names == 'PDCD1']
CD274_on = adata_sample_tpm.X[adata_sample_tpm.obs['timepoint'] == 'On', adata_sample_tpm.var_names == 'CD274']
CTLA4_on = adata_sample_tpm.X[adata_sample_tpm.obs['timepoint'] == 'On', adata_sample_tpm.var_names == 'CTLA4']
GZMK_on = adata_sample_tpm.X[adata_sample_tpm.obs['timepoint'] == 'On', adata_sample_tpm.var_names == 'GZMK']

In [ ]:
# plot scatter plot for pre 
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axes[0].scatter(CD274_on, cxcl13_on, s=2, color='black')
axes[0].set_xlabel('CD274')
axes[0].set_ylabel('CXCL13')
axes[1].scatter(pdcd1_on, cxcl13_on, s=2, color='black')
axes[1].set_xlabel('PDCD1')
axes[2].scatter(pdcd1_on * CD274_on, cxcl13_on, s=2, color='black' )
axes[2].set_xlabel('PDCD1 * CD274')
fig.tight_layout()


In [ ]:
# plot scatter plot for pre 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
axes[0].scatter(pdcd1_pre * CD274_pre, cxcl13_pre, s=2, color='black' )
axes[0].set_xlabel('PDCD1 * CD274')
axes[1].scatter(pdcd1_on * CD274_on, cxcl13_on, s=2, color='black' )
axes[1].set_xlabel('PDCD1 * CD274')
fig.tight_layout()
plt.show()